In [1]:
import json
import pandas as pd
from datetime import datetime
from listas import charlson_dict, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import (obtenir_data_presencia_codi, restar_dates, codis_ICD, nombre_ingressos,
                                 dies_ingressat_total, interval_10_edat,
                                 sumar_barthel, sumar_emina, obtenir_ultima_clau, obtenir_pes_o_mitjana,
                                 canadenca_comparada, disfagia_mecvvs, extreure_valors_claus,
                                 extreure_valors_claus_simple, obtenir_valors_clau_interes, index_charlson,
                                 obtenir_pes_mes_antic, obtenir_pes_mes_nou, obtenir_data_pes_mes_antic,
                                 obtenir_primera_data_mecvv, obtenir_pes_coincident_mecvv, restar_columnes_object)

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [2]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


In [3]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)]
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) &
           (taula['P diagnosticada'] == 1.0)]
TOTAL = taula

# Síndromes geriàtrics

## Index de Charlson

In [4]:
Mitjana_charlson = AMB_PA['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 10.24 ± 10.73


In [5]:
Mitjana_charlson = AMB_PA_MECVV['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA_MECVV['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.00 ± 11.60


In [6]:
Mitjana_charlson = SENSE_PA['Charlson'].mean()
desviacio_estandard_charlson = SENSE_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.50 ± 12.83


In [7]:
Mitjana_charlson = TOTAL['Charlson'].mean()
desviacio_estandard_charlson = TOTAL['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 11.62 ± 13.16


#### Pvalor

##### Normalitat

In [8]:
stat, p_value = shapiro(AMB_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7994
Valor p = 0.0020
El nombre de files no segueix una distribució normal.


In [9]:
stat, p_value = shapiro(AMB_PA_MECVV['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9651
Valor p = 0.8571
El nombre de files segueix una distribució normal


In [10]:
stat, p_value = shapiro(SENSE_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9177
Valor p = 0.4113
El nombre de files segueix una distribució normal


In [11]:
stat, p_value = shapiro(TOTAL['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7836
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [12]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Charlson'], AMB_PA_MECVV['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.23085687112868614


In [13]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Charlson'], SENSE_PA['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.1977214805403812


In [14]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['Charlson'], SENSE_PA['Charlson'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.9360013644802581


In [15]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.8623665948573777


In [16]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.2016616772763471


In [17]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.19370672249585386


## Caigudes prèvies

In [18]:
caprev_AMB_PA= (AMB_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [19]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [20]:
caprev_SENSE_PA= (SENSE_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

Percentatge de caigudes prèvies: 12.50%


In [21]:
caprev_TOTAL= (TOTAL['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_TOTAL:.2f}%")

Percentatge de caigudes prèvies: 6.00%


#### Pvalor

##### Normalitat

In [22]:
stat, p_value = shapiro(AMB_PA['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [23]:
stat, p_value = shapiro(AMB_PA_MECVV['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [24]:
stat, p_value = shapiro(SENSE_PA['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [25]:
stat, p_value = shapiro(TOTAL['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2548
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [26]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=AMB_PA_MECVV['Caigudes prèvies'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.


In [27]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.


In [28]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [ ]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")

In [29]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.


In [30]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.


## Deliris

In [31]:
del_AMB_PA= (AMB_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

Percentatge de deliris: 23.53%


In [32]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

Percentatge de deliris: 25.00%


In [33]:
del_SENSE_PA= (SENSE_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

Percentatge de deliris: 62.50%


In [34]:
del_TOTAL= (TOTAL['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_TOTAL:.2f}%")

Percentatge de deliris: 20.00%


#### Pvalor

##### Normalitat

In [35]:
stat, p_value = shapiro(AMB_PA['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [36]:
stat, p_value = shapiro(AMB_PA_MECVV['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [37]:
stat, p_value = shapiro(SENSE_PA['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [38]:
stat, p_value = shapiro(TOTAL['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [39]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Deliris'], columns=AMB_PA_MECVV['Deliris'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.


In [40]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Deliris'], columns=SENSE_PA['Deliris'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.


In [41]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=SENSE_PA['Deliris'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [42]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.


In [43]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=TOTAL['Deliris'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.


In [44]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.


## Demència

In [45]:
dem_AMB_PA= (AMB_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA:.2f}%")

Percentatge de demència: 47.06%


In [46]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA_MECVV:.2f}%")

Percentatge de demència: 25.00%


In [47]:
dem_SENSE_PA= (SENSE_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_SENSE_PA:.2f}%")

Percentatge de demència: 62.50%


In [48]:
dem_TOTAL= (TOTAL['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_TOTAL:.2f}%")

Percentatge de demència: 32.00%


#### Pvalor

##### Normalitat

In [49]:
stat, p_value = shapiro(AMB_PA['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [50]:
stat, p_value = shapiro(AMB_PA_MECVV['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [51]:
stat, p_value = shapiro(SENSE_PA['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [52]:
stat, p_value = shapiro(TOTAL['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5879
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [53]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Demència'], columns=AMB_PA_MECVV['Demència'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.


In [54]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Demència'], columns=SENSE_PA['Demència'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.


In [55]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=SENSE_PA['Demència'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [56]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Demència'], columns=TOTAL['Demència'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.


In [57]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=TOTAL['Demència'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.


In [58]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Demència'], columns=TOTAL['Demència'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.


## Síndrome depressiu

In [59]:
depre_AMB_PA= (AMB_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [60]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [61]:
depre_SENSE_PA= (SENSE_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [62]:
depre_TOTAL= (TOTAL['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_TOTAL:.2f}%")

Percentatge de síndrome depressiu: 14.00%


#### Pvalor

##### Normalitat

In [63]:
stat, p_value = shapiro(AMB_PA['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [64]:
stat, p_value = shapiro(AMB_PA_MECVV['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [65]:
stat, p_value = shapiro(SENSE_PA['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [66]:
stat, p_value = shapiro(TOTAL['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4124
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [67]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=AMB_PA_MECVV['Síndrome depressiu'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.


In [68]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.


In [69]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [70]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.


In [71]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.


In [72]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.


## Incontinència urinària

In [73]:
ic_AMB_PA= (AMB_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA:.2f}%")

Percentatge d'incontinència urinària: 29.41%


In [74]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència urinària: 50.00%


In [75]:
ic_SENSE_PA= (SENSE_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_SENSE_PA:.2f}%")

Percentatge d'incontinència urinària: 0.00%


In [76]:
ic_TOTAL= (TOTAL['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_TOTAL:.2f}%")

Percentatge d'incontinència urinària: 16.00%


#### Pvalor

##### Normalitat

In [77]:
stat, p_value = shapiro(AMB_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [78]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [79]:
stat, p_value = shapiro(SENSE_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [80]:
stat, p_value = shapiro(TOTAL['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4412
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [81]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=AMB_PA_MECVV['Incont.uri'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [82]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [83]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [84]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.


In [85]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.


In [86]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.


## Incontinència fecal 

In [87]:
ifec_AMB_PA= (AMB_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA:.2f}%")

Percentatge d'incontinència fecal: 29.41%


In [88]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència fecal: 25.00%


In [89]:
ifec_SENSE_PA= (SENSE_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_SENSE_PA:.2f}%")

Percentatge d'incontinència fecal: 0.00%


In [90]:
ifec_TOTAL= (TOTAL['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_TOTAL:.2f}%")

Percentatge d'incontinència fecal: 12.00%


#### Pvalor

##### Normalitat

In [91]:
stat, p_value = shapiro(AMB_PA['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [92]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [93]:
stat, p_value = shapiro(SENSE_PA['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [94]:
stat, p_value = shapiro(TOTAL['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3802
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [95]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=AMB_PA_MECVV['Incont.fec'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.


In [96]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.


In [97]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_23, p_value_23, dof_23, expected_13 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [98]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.


In [99]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.


In [100]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.


## Úlceres de pressió

In [101]:
up_AMB_PA= (AMB_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [102]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [103]:
up_SENSE_PA= (SENSE_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [104]:
up_TOTAL= (TOTAL['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_TOTAL:.2f}%")

Percentatge de úlceres de pressió: 2.00%


#### Pvalor

##### Normalitat

In [105]:
stat, p_value = shapiro(AMB_PA['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [106]:
stat, p_value = shapiro(AMB_PA_MECVV['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [107]:
stat, p_value = shapiro(SENSE_PA['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [108]:
stat, p_value = shapiro(TOTAL['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1254
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [109]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=AMB_PA_MECVV['Úlceres pressió'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")


Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.


In [110]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")


Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.


In [309]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [112]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")


Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.


In [113]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")


Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.


In [114]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")


Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.


## Immobilitat

In [115]:
imm_AMB_PA= (AMB_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [296]:
imm_AMB_PA_MECVV= (AMB_PA_MECVV['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA_MECVV:.2f}%")

Percentatge d'immobilitat : 0.00%


In [297]:
imm_SENSE_PA= (SENSE_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_SENSE_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [298]:
imm_TOTAL= (TOTAL['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_TOTAL:.2f}%")

Percentatge d'immobilitat : 0.00%


#### Pvalor

##### Normalitat

In [299]:
stat, p_value = shapiro(AMB_PA['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [300]:
stat, p_value = shapiro(AMB_PA_MECVV['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [301]:
stat, p_value = shapiro(SENSE_PA['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [303]:
stat, p_value = shapiro(TOTAL['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [304]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=AMB_PA_MECVV['Immobilitat']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.


In [305]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=SENSE_PA['Immobilitat']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.


In [308]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Immobilitat'], columns=SENSE_PA['Immobilitat']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i SENSE_PA.") 


ValueError: No data; `observed` has size 0.

In [307]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.


In [310]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.


In [311]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.


## Confusió

In [312]:
conf_AMB_PA= (AMB_PA['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA:.2f}%")

Percentatge de confusió : 23.53%


In [313]:
conf_AMB_PA_MECVV= (AMB_PA_MECVV['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA_MECVV:.2f}%")

Percentatge de confusió : 25.00%


In [314]:
conf_SENSE_PA= (SENSE_PA['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_SENSE_PA:.2f}%")

Percentatge de confusió : 62.50%


In [315]:
conf_TOTAL= (TOTAL['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_TOTAL:.2f}%")

Percentatge de confusió : 20.00%


#### Pvalor

##### Normalitat

In [316]:
stat, p_value = shapiro(AMB_PA['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [317]:
stat, p_value = shapiro(AMB_PA_MECVV['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [318]:
stat, p_value = shapiro(SENSE_PA['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [319]:
stat, p_value = shapiro(TOTAL['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [320]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Confusió'], columns=AMB_PA_MECVV['Confusió']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.


In [321]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Confusió'], columns=SENSE_PA['Confusió']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.


In [322]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Confusió'], columns=SENSE_PA['Confusió']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [323]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.


In [325]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.


In [326]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.


## Osteoporosi

In [327]:
ost_AMB_PA= (AMB_PA['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [329]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [330]:
ost_SENSE_PA= (SENSE_PA['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_SENSE_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [331]:
ost_TOTAL= (TOTAL['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_TOTAL:.2f}%")

Percentatge d'osteoporosi: 0.00%


#### Pvalor

##### Normalitat

In [332]:
stat, p_value = shapiro(AMB_PA['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [333]:
stat, p_value = shapiro(AMB_PA_MECVV['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [334]:
stat, p_value = shapiro(SENSE_PA['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [335]:
stat, p_value = shapiro(TOTAL['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [336]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=AMB_PA_MECVV['Osteoporosi']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.


In [337]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=SENSE_PA['Osteoporosi']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.


In [338]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Osteoporosi'], columns=SENSE_PA['Osteoporosi']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi  en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [339]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.


In [340]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.


In [341]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.


## Sarcopènia

In [342]:
sarc_AMB_PA= (AMB_PA['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

Percentatge de sarcopènia: 0.00%


In [343]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

Percentatge de sarcopènia: 0.00%


In [344]:
sarc_SENSE_PA= (SENSE_PA['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

Percentatge de sarcopènia: 12.50%


In [345]:
sarc_TOTAL= (TOTAL['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_TOTAL:.2f}%")

Percentatge de sarcopènia: 4.00%


#### Pvalor

##### Normalitat

In [346]:
stat, p_value = shapiro(AMB_PA['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [347]:
stat, p_value = shapiro(AMB_PA_MECVV['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [348]:
stat, p_value = shapiro(SENSE_PA['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [349]:
stat, p_value = shapiro(TOTAL['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1982
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [351]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=AMB_PA_MECVV['Sarcopènia']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.


In [352]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=SENSE_PA['Sarcopènia']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia AMB_PA i SENSE_PA.


In [353]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Sarcopènia'], columns=SENSE_PA['Sarcopènia']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [354]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.


In [355]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.


In [356]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.


## Trastorns del son

In [360]:
ts_AMB_PA= (AMB_PA['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [358]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

Percentatge de trastorns del son: 0.00%


In [359]:
ts_SENSE_PA= (SENSE_PA['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [361]:
ts_TOTAL= (TOTAL['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_TOTAL:.2f}%")

Percentatge de trastorns del son: 0.00%


#### Pvalor

##### Normalitat

In [362]:
stat, p_value = shapiro(AMB_PA['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [363]:
stat, p_value = shapiro(AMB_PA_MECVV['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [364]:
stat, p_value = shapiro(SENSE_PA['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [365]:
stat, p_value = shapiro(TOTAL['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [366]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Probl.son'], columns=AMB_PA_MECVV['Probl.son']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.


In [367]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Probl.son'], columns=SENSE_PA['Probl.son']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.


In [368]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Probl.son'], columns=SENSE_PA['Probl.son']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [369]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.


In [370]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.


In [371]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.


## Dolor crònic

In [372]:
dc_AMB_PA= (AMB_PA['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [373]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

Percentatge de dolor crònic: 0.00%


In [374]:
dc_SENSE_PA= (SENSE_PA['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [375]:
dc_TOTAL= (TOTAL['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_TOTAL:.2f}%")

Percentatge de dolor crònic: 0.00%


#### Pvalor

##### Normalitat

In [376]:
stat, p_value = shapiro(AMB_PA['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [377]:
stat, p_value = shapiro(AMB_PA_MECVV['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [381]:
stat, p_value = shapiro(SENSE_PA['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [382]:
stat, p_value = shapiro(TOTAL['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [383]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=AMB_PA_MECVV['Dolor crònic']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.


In [384]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=SENSE_PA['Dolor crònic']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.


In [385]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Dolor crònic'], columns=SENSE_PA['Dolor crònic']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [386]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.


In [388]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.


In [389]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.


## Iatrogènic

In [390]:
iatr_AMB_PA= (AMB_PA['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [391]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

Percentatge de iatrogènic: 0.00%


In [392]:
iatr_SENSE_PA= (SENSE_PA['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [393]:
iatr_TOTAL= (TOTAL['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_TOTAL:.2f}%")

Percentatge de iatrogènic: 0.00%


#### Pvalor

##### Normalitat

In [394]:
stat, p_value = shapiro(AMB_PA['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [395]:
stat, p_value = shapiro(AMB_PA_MECVV['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [396]:
stat, p_value = shapiro(SENSE_PA['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [397]:
stat, p_value = shapiro(TOTAL['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [398]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=AMB_PA_MECVV['Iatrogènic']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.


In [399]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=SENSE_PA['Iatrogènic']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.


In [400]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Iatrogènic'], columns=SENSE_PA['Iatrogènic']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [401]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.


In [402]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.


In [403]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.


## Restrenyiment

In [404]:
est_AMB_PA= (AMB_PA['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [405]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

Percentatge de restrenyiment: 0.00%


In [406]:
est_SENSE_PA= (SENSE_PA['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [407]:
est_TOTAL= (TOTAL['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_TOTAL:.2f}%")

Percentatge de restrenyiment: 0.00%


#### Pvalor

##### Normalitat

In [408]:
stat, p_value = shapiro(AMB_PA['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [409]:
stat, p_value = shapiro(AMB_PA_MECVV['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [410]:
stat, p_value = shapiro(SENSE_PA['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [411]:
stat, p_value = shapiro(TOTAL['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [412]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=AMB_PA_MECVV['Restrenyiment']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.


In [413]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=SENSE_PA['Restrenyiment']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.


In [414]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Restrenyiment'], columns=SENSE_PA['Restrenyiment']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [415]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.


In [416]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.


In [417]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.


# Historial mèdic/comorbilitats (aqui)

## Malalties CV

In [186]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [187]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

Percentatge de problemes CV: 0.00%


In [188]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

Percentatge de problemes CV: 25.00%


#### Pvalor

##### Normalitat 

In [189]:
stat, p_value = shapiro(AMB_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [190]:
stat, p_value = shapiro(AMB_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [191]:
stat, p_value = shapiro(SENSE_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [192]:
stat, p_value_12 = mannwhitneyu(AMB_PA['CV'], AMB_PA_MECVV['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.5483429354628137


In [193]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.19610818609418723


In [194]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.1699515583739133


## Malalties del cor

In [195]:
cora_AMB_PA= (AMB_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA:.2f}%")

KeyError: 'probl_corazon'

In [196]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_AMB_PA_MECVV:.2f}%")

KeyError: 'probl_corazon'

In [197]:
cora_SENSE_PA= (SENSE_PA['probl_corazon'] == 1.0).mean() * 100
print(f"Percentatge de malalties de cor: {cora_SENSE_PA:.2f}%")

KeyError: 'probl_corazon'

#### Pvalor

##### Normalitat

In [198]:
stat, p_value = shapiro(AMB_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'probl_corazon'

In [199]:
stat, p_value = shapiro(AMB_PA_MECVV['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'probl_corazon'

In [200]:
stat, p_value = shapiro(SENSE_PA['probl_corazon'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'probl_corazon'

In [201]:
stat, p_value_12 = mannwhitneyu(AMB_PA['probl_corazon'], AMB_PA_MECVV['probl_corazon'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'probl_corazon'

In [202]:
stat, p_value_13 = mannwhitneyu(AMB_PA['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.19610818609418723


In [203]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['CV'], SENSE_PA['CV'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.1699515583739133


## Malalties neurodegeneratives

In [204]:
nd_AMB_PA= (AMB_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

KeyError: 'neurodegenerativas'

In [205]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

KeyError: 'neurodegenerativas'

In [206]:
nd_SENSE_PA= (SENSE_PA['neurodegenerativas'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

KeyError: 'neurodegenerativas'

#### Pvalor

##### Normalitat

In [207]:
stat, p_value = shapiro(AMB_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neurodegenerativas'

In [208]:
stat, p_value = shapiro(AMB_PA_MECVV['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neurodegenerativas'

In [209]:
stat, p_value = shapiro(SENSE_PA['neurodegenerativas'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neurodegenerativas'

In [210]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neurodegenerativas'], AMB_PA_MECVV['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'neurodegenerativas'

In [211]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'neurodegenerativas'

In [212]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neurodegenerativas'], SENSE_PA['neurodegenerativas'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'neurodegenerativas'

## Diabetes mellitus

In [213]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA:.2f}%")

Percentatge de diabetes mellitus: 23.53%


In [214]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_AMB_PA_MECVV:.2f}%")

Percentatge de diabetes mellitus: 25.00%


In [215]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetes mellitus: {dm_SENSE_PA:.2f}%")

Percentatge de diabetes mellitus: 25.00%


#### Pvalor 

##### Normalitat

In [216]:
stat, p_value = shapiro(AMB_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [217]:
stat, p_value = shapiro(AMB_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [218]:
stat, p_value = shapiro(SENSE_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [219]:
stat, p_value_12 = mannwhitneyu(AMB_PA['DM'], AMB_PA_MECVV['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.9686157491081862


In [220]:
stat, p_value_13 = mannwhitneyu(AMB_PA['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PAN", p_value_13)

P-valor entre AMB_PA i SENSE_PAN 0.9686157491081862


In [221]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['DM'], SENSE_PA['DM'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PAN", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PAN 1.0


## Hepatopaties

In [222]:
hp_AMB_PA= (AMB_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

KeyError: 'hepatopatias'

In [223]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

KeyError: 'hepatopatias'

In [224]:
hp_SENSE_PA= (SENSE_PA['hepatopatias'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

KeyError: 'hepatopatias'

#### Pvalor

##### Normalitat

In [225]:
stat, p_value = shapiro(AMB_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [226]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [227]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [228]:
stat, p_value_12 = mannwhitneyu(AMB_PA['hepatopatias'], AMB_PA_MECVV['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'hepatopatias'

In [229]:
stat, p_value_13 = mannwhitneyu(AMB_PA['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'hepatopatias'

In [230]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['hepatopatias'], SENSE_PA['hepatopatias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'hepatopatias'

## Neoplàsia

In [231]:
neopl_AMB_PA= (AMB_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA:.2f}%")

KeyError: 'neoplasias'

In [232]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_AMB_PA_MECVV:.2f}%")

KeyError: 'neoplasias'

In [233]:
neopl_SENSE_PA= (SENSE_PA['neoplasias'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsia: {neopl_SENSE_PA:.2f}%")

KeyError: 'neoplasias'

#### Pvalor

##### Normalitat

In [234]:
stat, p_value = shapiro(AMB_PA['neoplasias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'neoplasias'

In [235]:
stat, p_value = shapiro(AMB_PA_MECVV['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [236]:
stat, p_value = shapiro(SENSE_PA['hepatopatias'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'hepatopatias'

In [237]:
stat, p_value_12 = mannwhitneyu(AMB_PA['neoplasias'], AMB_PA_MECVV['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'neoplasias'

In [238]:
stat, p_value_13 = mannwhitneyu(AMB_PA['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_12)

KeyError: 'neoplasias'

In [239]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['neoplasias'], SENSE_PA['neoplasias'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'neoplasias'

## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [240]:
AMB_PA.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA['creatinina'], errors='coerce')
AMB_PA_MECVV.loc[:, 'creatinina'] = pd.to_numeric(AMB_PA_MECVV['creatinina'], errors='coerce')
SENSE_PA.loc[:, 'creatinina'] = pd.to_numeric(SENSE_PA['creatinina'], errors='coerce')

KeyError: 'creatinina'

In [241]:
ecr_AMB_PA= (AMB_PA['creatinina'] >= 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

KeyError: 'creatinina'

In [242]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

KeyError: 'creatinina'

In [243]:
ecr_SENSE_PA= (SENSE_PA['creatinina'] > 1.5).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

KeyError: 'creatinina'

#### Pvalor

##### Normalitat

In [244]:
stat, p_value = shapiro(AMB_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'creatinina'

In [245]:
stat, p_value = shapiro(AMB_PA_MECVV['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'creatinina'

In [246]:
stat, p_value = shapiro(SENSE_PA['creatinina'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'creatinina'

In [247]:
stat, p_value_12 = ttest_ind(AMB_PA['creatinina'], AMB_PA_MECVV['creatinina'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'creatinina'

In [248]:
stat, p_value_13 = mannwhitneyu(AMB_PA['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'creatinina'

In [249]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['creatinina'], SENSE_PA['creatinina'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'creatinina'

## Fallida renal aguda (o insuficiència renal aguda)

In [250]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [251]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [252]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

Percentatge de fallida renal aguda: 50.00%


#### Pvalor

##### Normalitat

In [253]:
stat, p_value = shapiro(AMB_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [254]:
stat, p_value = shapiro(AMB_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [255]:
stat, p_value = shapiro(SENSE_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [256]:
stat, p_value_12 = mannwhitneyu(AMB_PA['ARF'], AMB_PA_MECVV['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.2073928860736831


In [257]:
stat, p_value_13 = mannwhitneyu(AMB_PA['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.2073928860736831


In [258]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['ARF'], SENSE_PA['ARF'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_23)

P-valor entre AMB_PA i SENSE_PA 1.0


## Marejos/síncope/lipotímia/reacció vasovagal

In [259]:
dizsyn_AMB_PA= (AMB_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

KeyError: 'mareos'

In [260]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

KeyError: 'mareos'

In [261]:
dizsyn_SENSE_PA= (SENSE_PA['mareos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

KeyError: 'mareos'

#### Pvalor

##### Normalitat

In [262]:
stat, p_value = shapiro(AMB_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'mareos'

In [263]:
stat, p_value = shapiro(AMB_PA_MECVV['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'mareos'

In [264]:
stat, p_value = shapiro(SENSE_PA['mareos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'mareos'

In [265]:
stat, p_value_12 = ttest_ind(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'mareos'

In [266]:
stat, p_value_12 = mannwhitneyu(AMB_PA['mareos'], AMB_PA_MECVV['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'mareos'

In [267]:
stat, p_value_13 = mannwhitneyu(AMB_PA['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'mareos'

In [268]:
stat, p_value_23 = mannwhitneyu(AMB_PA_MECVV['mareos'], SENSE_PA['mareos'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'mareos'

## VIH

In [269]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

Percentatge de VIH: 0.00%


In [270]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [271]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

Percentatge de VIH: 0.00%


#### Pvalor

##### Normalitat

In [272]:
stat, p_value = shapiro(AMB_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [273]:
stat, p_value = shapiro(AMB_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [274]:
stat, p_value = shapiro(SENSE_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [275]:
stat, p_value_12 = ttest_ind(AMB_PA['VIH'], AMB_PA_MECVV['VIH'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV nan


In [276]:
stat, p_value_13 = ttest_ind(AMB_PA['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA nan


In [277]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['VIH'], SENSE_PA['VIH'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA nan


## Psicosi

In [278]:
psi_AMB_PA= (AMB_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

KeyError: 'psicosis'

In [279]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

KeyError: 'psicosis'

In [280]:
psi_SENSE_PA= (SENSE_PA['psicosis'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

KeyError: 'psicosis'

#### Pvalor 

##### Normalitat

In [281]:
stat, p_value = shapiro(AMB_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'psicosis'

In [282]:
stat, p_value = shapiro(AMB_PA_MECVV['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'psicosis'

In [283]:
stat, p_value = shapiro(SENSE_PA['psicosis'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'psicosis'

In [284]:
stat, p_value_12 = ttest_ind(AMB_PA['psicosis'], AMB_PA_MECVV['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'psicosis'

In [285]:
stat, p_value_13 = ttest_ind(AMB_PA['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'psicosis'

In [286]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['psicosis'], SENSE_PA['psicosis'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'psicosis'

## Deficiències nutricionals

In [287]:
nutri_AMB_PA= (AMB_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

KeyError: 'def_nutri'

In [288]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

KeyError: 'def_nutri'

In [289]:
nutri_SENSE_PA= (SENSE_PA['def_nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

KeyError: 'def_nutri'

#### Pvalor

##### Normalitat

In [290]:
stat, p_value = shapiro(AMB_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'def_nutri'

In [291]:
stat, p_value = shapiro(AMB_PA_MECVV['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'def_nutri'

In [292]:
stat, p_value = shapiro(SENSE_PA['def_nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

KeyError: 'def_nutri'

In [293]:
stat, p_value_12 = ttest_ind(AMB_PA['def_nutri'], AMB_PA_MECVV['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

KeyError: 'def_nutri'

In [294]:
stat, p_value_13 = ttest_ind(AMB_PA['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

KeyError: 'def_nutri'

In [295]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['def_nutri'], SENSE_PA['def_nutri'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

KeyError: 'def_nutri'